In [1]:
import os
import pandas as pd
import numpy as np
import pandas_datareader
from pandas_datareader import data as pdr
from datetime import datetime, timedelta
import yfinance as yf

def stockData(sList, stDate, endDate):
    """Take list of stocks and pull data for that stock and create technical indicator variables
    sList requires list of stocks, dates should be entered in format 'YYYY-MM-DD' """
   
    data = []
    for element in sList:
        try:
            temp = pdr.get_data_yahoo(element, start = stDate, end = endDate)
            #add an indentifier
            temp['Symbol'] = element
            temp['Close'] = temp['Close']
        except: 
            #If stock symbol cannot be found make it $1
            print (element)
            temp['Symbol'] = element
            temp['Close'] = 1
        else:
        #add data to list 
            data.append(temp)
        #convert to dataframe    
        df = pd.concat(data)
        #drop rows for which we will not have all data points calculated
        df = df.dropna()
        #dates above what user entered
        df = df.loc[stDate : endDate]
        #order and limit columns to those needed
        df = df.loc[:, ['Symbol', 'Close']]
        df['Day'] = pd.to_datetime(df.index)

    
    return(df)


In [2]:
#set working directory 
os.chdir("/Users/bkrei/Desktop/Bk's Stuff Desktop/School/Github NEW PATH/Portfolio-Challenge")

In [3]:
#Read in individual portfolios
portfolios = pd.read_csv("https://raw.githubusercontent.com/bkreis84/Portfolio-Challenge/master/Portfolios.csv")

In [4]:
#List of all ticker symbols
symbols = portfolios['Symbol'].drop_duplicates()
symbols.head(300)

0       AAPL
1      AGTHX
2       AMZN
3        CHE
4       DOCU
5      GSGRX
6       ICLN
7      OEGAX
8       PBFS
9        QQQ
10       SPY
13     BRK-B
14     CMCSA
15       CVS
16       DIS
17        GE
18       JNJ
19       JPM
20      NFLX
22      SBUX
23       XOM
28        BA
29         C
30      CHWY
31      CVRS
32      DELL
33       IAG
37       BAM
38       BEP
39       CGC
       ...  
371     ZUMZ
376     EGRX
381     ACER
382       KR
383     UUUU
384     ZIXI
390      MTZ
397      NOC
399      TPL
401     ANET
402     INGN
403      MED
404     SIVB
405     ALXN
407      RTN
409     ACST
411      DNR
413       NM
423       JP
426        X
441     CELG
442     COUP
443     CTSH
449     FTNT
450      HON
473      SAM
477      CXO
478     GWPH
480     MOMO
482     RVNC
Name: Symbol, Length: 235, dtype: object

In [5]:
symbols.to_csv("symbols.csv", header = False)

In [6]:
#Pass the ticker list to the function, print invalid stock symbols and create dataframe with intial prices
beg_prices = stockData(list(symbols), '2019-09-06', '2019-09-06')
beg_prices.shape

CASHPIO


(234, 3)

In [8]:
#Pull SNAP which for some reason is the only stock with an issue.
er = stockData(['SNAP'], '2019-09-06', '2019-09-06')

In [9]:
#Combine the main and SNAP dataframes
frames = [beg_prices, er]
beg_prices = pd.concat(frames)

In [11]:
beg_prices.to_csv("beg_prices.csv")

In [12]:
beg_prices = pd.read_csv("https://raw.githubusercontent.com/bkreis84/Portfolio-Challenge/master/beg_prices.csv")
combined = pd.merge(portfolios, beg_prices, how = 'left', on = "Symbol", validate="many_to_one")

In [13]:
combined.head(5)

,Name,Symbol,% of Portfolio,Cost Basis,Date,Close,Day
0,Adrienne Villano,AAPL,0.07,14000,2019-09-06,213.259995,2019-09-06
1,Adrienne Villano,AGTHX,0.11,22000,2019-09-06,50.000000,2019-09-06
2,Adrienne Villano,AMZN,0.07,14000,2019-09-06,1833.510010,2019-09-06
3,Adrienne Villano,CHE,0.18,36000,2019-09-06,439.820007,2019-09-06
4,Adrienne Villano,DOCU,0.05,10000,2019-09-06,56.270000,2019-09-06


In [14]:
#Calc # of shares purchased
combined["# of Shares"] = combined["Cost Basis"]/ combined["Close"]
#Convert names to proper case
#combined['Name'] = list(map(lambda x: x.title(), combined['Name']))
#combined.head()

In [15]:
combined.to_csv("initial portfolios.csv")

# Simple Comparison (Initial vs Current Value) START HERE TO UPDATE

In [30]:
#Select date to compare to initial prices
cur_prices = stockData(list(symbols), '2019-09-26', '2019-09-26')

TSS
CASHPIO
SKIS


In [31]:
#Pull SNAP which for some reason is the only stock with an issue.
er2 =  stockData(['SNAP'], '2019-09-26', '2019-09-26')

In [32]:
#Combine the main and SNAP dataframes
frames2 = [cur_prices, er2]
cur_prices = pd.concat(frames2)

In [33]:
cur_prices.sort_values("Symbol", inplace = True)
cur_prices.drop_duplicates(subset = "Symbol", inplace = True)

In [34]:
cur_prices.to_csv("cur_prices.csv")

In [35]:
cur_prices= pd.read_csv("https://raw.githubusercontent.com/bkreis84/Portfolio-Challenge/master/cur_prices.csv")
cur_prices.head()


,Date,Symbol,Close,Day
0,2019-09-26,AAL,27.190001,2019-09-26
1,2019-09-26,ACB,4.740000,2019-09-26
2,2019-09-26,ACER,3.190000,2019-09-26
3,2019-09-26,ACST,1.890000,2019-09-26
4,2019-09-26,ADBE,283.809998,2019-09-26


In [36]:
detail = pd.merge(combined, cur_prices, how = 'left', on = "Symbol", validate="many_to_one")

In [37]:
detail.head()

,Name,Symbol,% of Portfolio,Cost Basis,Date_x,Close_x,Day_x,# of Shares,Date_y,Close_y,Day_y
0,Adrienne Villano,AAPL,0.07,14000,2019-09-06,213.259995,2019-09-06,65.647568,NaN,NaN,NaN
1,Adrienne Villano,AGTHX,0.11,22000,2019-09-06,50.000000,2019-09-06,440.000000,2019-09-26,1.000000,2019-09-26
2,Adrienne Villano,AMZN,0.07,14000,2019-09-06,1833.510010,2019-09-06,7.635628,2019-09-26,1739.839966,2019-09-26
3,Adrienne Villano,CHE,0.18,36000,2019-09-06,439.820007,2019-09-06,81.851665,2019-09-26,417.649994,2019-09-26
4,Adrienne Villano,DOCU,0.05,10000,2019-09-06,56.270000,2019-09-06,177.714589,NaN,NaN,NaN


In [38]:
detail['Current_Value'] = detail['Close_y'] * detail['# of Shares']
detail['Perc_Return'] = detail['Close_y']/detail['Close_x']  -1
detail.head(5)

,Name,Symbol,% of Portfolio,Cost Basis,Date_x,Close_x,Day_x,# of Shares,Date_y,Close_y,Day_y,Current_Value,Perc_Return
0,Adrienne Villano,AAPL,0.07,14000,2019-09-06,213.259995,2019-09-06,65.647568,NaN,NaN,NaN,NaN,NaN
1,Adrienne Villano,AGTHX,0.11,22000,2019-09-06,50.000000,2019-09-06,440.000000,2019-09-26,1.000000,2019-09-26,440.000000,-0.980000
2,Adrienne Villano,AMZN,0.07,14000,2019-09-06,1833.510010,2019-09-06,7.635628,2019-09-26,1739.839966,2019-09-26,13284.770409,-0.051088
3,Adrienne Villano,CHE,0.18,36000,2019-09-06,439.820007,2019-09-06,81.851665,2019-09-26,417.649994,2019-09-26,34185.347483,-0.050407
4,Adrienne Villano,DOCU,0.05,10000,2019-09-06,56.270000,2019-09-06,177.714589,NaN,NaN,NaN,NaN,NaN


In [39]:
detail.to_csv("detail.csv")

### Individual Performance

In [40]:
ranking = detail.groupby("Name")['Cost Basis', 'Current_Value'].sum().reset_index().sort_values("Current_Value", ascending=False)

In [41]:
ranking['Perc_Return'] = ranking['Current_Value']/ranking['Cost Basis'] -1
ranking['Rank'] = ranking['Perc_Return'].rank(ascending=False)
ranking.head(25)

,Name,Cost Basis,Current_Value,Perc_Return,Rank
47,Robert Kovacs,200000,222425.641226,0.112128,1.0
29,Kelly Nicholson,200000,209844.456116,0.049222,2.0
40,Nick Capullo,200000,204190.360137,0.020952,3.0
19,Elise Shaver,200000,202405.078427,0.012025,4.0
42,Pete Lindner,200000,202218.070438,0.011090,5.0
30,Kimberly Matzen,200000,201896.293033,0.009481,6.0
54,Trudy Seeber,200000,201794.599095,0.008973,7.0
7,Barb Derian,200000,201144.430136,0.005722,8.0
9,Bradley Bonfante,200000,200901.003604,0.004505,9.0
13,Dan Cancelliere,200000,200476.418884,0.002382,10.0


In [42]:
ranking.to_csv("individual rankings.csv")

### Best Individual Stock Returns

In [43]:
best_stocks = detail.drop_duplicates(['Symbol']).reset_index().sort_values(['Perc_Return'], ascending=False)

In [44]:
best_stocks = best_stocks.loc[:, ['Symbol', 'Perc_Return']]
best_stocks.head(10)

,Symbol,Perc_Return
79,PIR,0.740260
165,HOV,0.516154
171,HOME,0.296961
234,RVNC,0.243790
148,ATLC,0.241915
106,GME,0.225058
182,DRRX,0.224719
203,FNMAL,0.194667
125,MARK,0.183673
97,PLUG,0.172566


In [45]:
best_stocks.to_csv("stock ranking.csv")

# Trend

In [ ]:
#Pull Only necessary columns
combined = combined.loc[:, ['Name', 'Symbol', '# of Shares']]

In [ ]:
Price_History = stockData(stocks['Ticker'], '2019-08-01', '2019-09-06')

In [ ]:
historical = pd.merge(combined, Price_History, how = 'outer', on = "Symbol", validate="many_to_many")


In [ ]:
historical['Value'] = historical['# of Shares'] * historical['Close']
historical.head()

In [ ]:
trend_data = historical.groupby(['Name', 'Day'])['Value'].sum().reset_index().sort_values(['Name', 'Day'])

In [ ]:
trend_data.head()

In [1]:
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode